# File and libraries

In [1]:
import pandas as pd
import numpy as np


file_tag = "Instacart Market Basket"

# DSLabs functions

In [2]:
%run "scripts/dslabs_functions.py"


# data functions

In [3]:
%run "scripts/data_functions.py"


data_functions lodaded


# sampling and testing

In [ ]:

test_data=True
# test_data=False


# Define the sampling function
def sample_user_orders(data, fraction=0.1):
    # Get unique user_ids and order_ids
    unique_user_orders = data[['user_id', 'order_id']].drop_duplicates()
    sampled_user_orders = unique_user_orders.sample(frac=fraction)
    
    # Filter the dataset to include only the sampled users and orders
    sampled_data = data[data[['user_id', 'order_id']].apply(tuple, axis=1).isin(sampled_user_orders.apply(tuple, axis=1))]
    return sampled_data


# Load the data
orders = pd.read_csv('data/input/orders.csv')


if test_data==True:



    # Apply the sampling to each group 1%
    sample=0.01
    order_data = sample_user_orders(orders, fraction=sample)
    

else:
    
    # Apply the sampling to each group 10%
    sample=0.1
    order_data = sample_user_orders(orders, fraction=sample)    
    
    
# drop eval_set as its not necessary
order_data=order_data.drop(['eval_set'], axis=1)

print(order_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 34211 entries, 19 to 3421036
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                34211 non-null  int64  
 1   user_id                 34211 non-null  int64  
 2   order_number            34211 non-null  int64  
 3   order_dow               34211 non-null  int64  
 4   order_hour_of_day       34211 non-null  int64  
 5   days_since_prior_order  32158 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 1.8 MB
None


## merge orders with prior

In [24]:
prior=pd.read_csv('data/input/order_products__prior.csv')


# merge the two dataframes on order id
data = pd.merge(prior,order_data , on='order_id', how='inner')

# class target column


In [25]:
target = "reordered"

values = data[target].value_counts(normalize=True) 
print(values)

reordered
1    0.591589
0    0.408411
Name: proportion, dtype: float64


In [13]:
data.shape

(325780, 9)

In [26]:
summary5 = data.describe(include="all")

summary5

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.218330e+05,321833.000000,321833.000000,321833.000000,321833.000000,321833.000000,321833.000000,321833.000000,301698.000000
mean,1.710249e+06,25551.097656,8.286148,0.591589,103150.460425,17.157507,2.723866,13.382173,11.131035
std,9.832156e+05,14085.925632,7.032177,0.491541,59845.033557,17.566541,2.081332,4.270336,8.805899
min,1.600000e+01,1.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000
25%,8.622320e+05,13512.000000,3.000000,0.000000,50986.000000,5.000000,1.000000,10.000000,5.000000
50%,1.710589e+06,25199.000000,6.000000,1.000000,103196.000000,11.000000,3.000000,13.000000,8.000000
75%,2.562917e+06,37836.000000,11.000000,1.000000,155396.000000,24.000000,5.000000,16.000000,15.000000
max,3.421075e+06,49688.000000,74.000000,1.000000,206208.000000,99.000000,6.000000,23.000000,30.000000


### additional date columns creation

In [32]:
# Categorize the time of day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

data['order_time_of_day'] = data['order_hour_of_day'].apply(categorize_time_of_day)

data['is_weekend'] = data['order_dow'].apply(lambda x: 1 if x >= 5 else 0)  # 1 for weekend, 0 for weekday


data['weeks_since_prior_order'] = data['days_since_prior_order'].apply(lambda x: round(x / 7, 0) if pd.notnull(x) else np.nan)



# enrich data

In [33]:


enriched_data = enrich_instacart_df(data)

                   
enriched_data.head(20)

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_time_of_day,is_weekend,weeks_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,16,9755,1,1,174840,18,3,12,13.0,Afternoon,0,2.0,Original Popcorn,23,19,popcorn jerky,snacks
1,16,25466,2,0,174840,18,3,12,13.0,Afternoon,0,2.0,Water,115,7,water seltzer sparkling water,beverages
2,16,45437,3,0,174840,18,3,12,13.0,Afternoon,0,2.0,Sea Salt Made With Organic Grain Rice Chips,107,19,chips pretzels,snacks
3,105,1194,1,0,178958,27,4,10,30.0,Morning,0,4.0,Natural Artisan Water,115,7,water seltzer sparkling water,beverages
4,105,12276,2,1,178958,27,4,10,30.0,Morning,0,4.0,Chocolate Chip Cookie Dough Ice Cream,37,1,ice cream ice,frozen
5,105,13176,3,1,178958,27,4,10,30.0,Morning,0,4.0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,105,27423,4,1,178958,27,4,10,30.0,Morning,0,4.0,Real Aged Cheddar Macaroni & Cheese,4,9,instant foods,dry goods pasta
7,105,17600,5,1,178958,27,4,10,30.0,Morning,0,4.0,"YoKids Squeezers Organic Low-Fat Yogurt, Straw...",120,16,yogurt,dairy eggs
8,105,4920,6,1,178958,27,4,10,30.0,Morning,0,4.0,Seedless Red Grapes,123,4,packaged vegetables fruits,produce
9,105,28204,7,1,178958,27,4,10,30.0,Morning,0,4.0,Organic Fuji Apple,24,4,fresh fruits,produce


# Final df to csv

In [34]:
data.to_csv('data/instacart_pre_proc.csv',index=False)